In [1]:
import pandas as pd
import numpy as np

In [2]:
def find(df,column,what,view=[], groupby="", sort=""):
    """
        trouve une valeur dans le tableau en fonction de la colonne
        
        groupby : mean, sum, first, last, size
        sort : ascending, descending
    """
    tmp = df[df[column].isin(what)]
    
    if groupby=="mean":
        tmp = tmp.groupby([column]).mean()
    elif groupby=="sum":
        tmp = tmp.groupby([column]).sum()
    elif  groupby=="first":
        tmp = tmp.groupby([column]).first()
    elif  groupby=="last":
        tmp = tmp.groupby([column]).last()
    elif  groupby=="size":
        tmp = tmp.groupby([column]).size()
        if view != []:
            print('\033[93m' + "GROUPBY Warn:"+ '\033[95m' +" pas de VIEW disponible avec l'option size")
            view = []
    
    if sort != "" and groupby != "":
        print('\033[93m' + "SORT Warn:"+ '\033[95m' +" inutile de trier les objets groupé")
        sort=""
    
    if sort=="ascending":
        tmp = tmp.sort_values(by=column, ascending=True)
    elif sort=="descending":
        tmp = tmp.sort_values(by=column, ascending=False)
            
    if view != [] :
       return tmp.loc[:,view]
    return tmp

In [3]:
def rangeValue(df,column,plage,view=[], groupby="first"):
    """
        regroupe les valeur celon une plage
        
        groupby : mean, sum, first, last, size 
            default -> first
    """
    tmp = df.groupby(pd.cut(csv[column], plage))
    
    if groupby=="mean":
        tmp = tmp.mean()
    elif groupby=="sum":
        tmp = tmp.sum()
    elif  groupby=="first":
        tmp = tmp.first()
    elif  groupby=="last":
        tmp = tmp.last()
    elif  groupby=="size":
        tmp = tmp.size()
        if view != []:
            print('\033[93m' + "GROUPBY Warn:"+ '\033[95m' +" pas de View disponible avec l'option size")
            view = []
            
    if view != [] :
       return tmp.loc[:,view]
    return tmp

In [7]:
list(range(0,5))

[0, 1, 2, 3, 4]

In [58]:
def changeValue(df, columns, origin, new, condition=[]):
    """
    change la ou les valeurs donnée dans origin par new dans le tableau 
    
    condition (str, object): assigne la valeur new si la condition est respecteer sinon elle garde la valeurs de  base
        origin ne sera pas pris en compte, il peut etre initialisé a ""
    condition ("around", int) : aroundi la valeur selon les colonnes selectionné, int correspond du nombre de chiffre apres la virgule
    """
    for i in range(len(columns)):
        if condition != () and len(condition) == 2:
            mode, form = condition
            if mode == ">":
                df[columns[i]] = df[columns[i]].apply(lambda x: new if x > form else x)
            elif mode == "<":
                df[columns[i]] = df[columns[i]].apply(lambda x: new if x < form else x)
            elif mode == "!=":
                df[columns[i]] = df[columns[i]].apply(lambda x: new if x != form else x)
            elif mode == "==":
                df[columns[i]] = df[columns[i]].apply(lambda x: new if x == form else x)
            elif mode == "<=":
                df[columns[i]] = df[columns[i]].apply(lambda x: new if x <= form else x)
            elif mode == ">=":
                df[columns[i]] = df[columns[i]].apply(lambda x: new if x >= form else x)
            elif mode == "around":
                df[columns[i]] = df[columns[i]].apply(lambda x: round(x, form) if type(x) is float else x)
        else:
            df[columns[i]] = df[columns[i]].apply(lambda x: new if x == origin else x)
                

In [56]:
csv = pd.read_csv('mower_market_snapshot.csv',delimiter=";")

In [10]:
find(csv,"warranty", ["3ans", "2ans"], view=["id"], groupby="size", sort="ascending")

GROUPBY Warn: pas de VIEW disponible avec l'option size
SORT Warn: inutile de trier les objets groupé


warranty
2ans    42
3ans    17
dtype: int64

In [11]:
rangeValue(csv, "price", [0, 200,300,600,float("inf")], groupby="mean")

,capacity,failure_rate,id,margin,price,market_share,attractiveness
price,,,,,,,
"(0.0, 200.0]",56.933206,0.220407,18948.346863,67.479837,74.193945,0.001225,0.626233
"(200.0, 300.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(300.0, 600.0]",38.632563,0.100551,18964.875300,263.736848,381.476375,0.002431,0.632159
"(600.0, inf]",25.351682,0.069466,18987.473373,566.520813,854.732719,0.005862,0.696661


In [28]:
find(csv, "product_type", ["electric", "electrique"], groupby="size")

product_type
electrique    413
dtype: int64

In [29]:
changeValue(csv, ["product_type"], "electrique", "electric")
find(csv, "product_type", ["electric", "electrique"], groupby="size")

product_type
electric    413
dtype: int64

In [57]:
changeValue(csv, ["price"], "", "", condition=("around", 2))
csv.head()

,capacity,failure_rate,id,margin,price,prod_cost,product_type,quality,warranty,market_share,attractiveness
0,10.492623,0.042528,18377,633.766584,828.80,195.03551928211797,auto-portee,Low,3ans,0.006360,0.650648
1,20.273730,0.037896,19363,731.576817,882.89,151.31809934886763,auto-portee,Low,3 ans.,0.006840,0.699792
2,20.190457,0.039111,19641,741.218202,919.36,178.13731921134948,auto-portee,Low,3_ans,0.006179,0.632129
3,27.220131,0.041303,18728,654.255635,817.09,162.82946157588628,auto-portee,Low,3 anss,0.007925,0.810730
4,16.370911,0.037938,18722,703.094527,844.58,141.48693796090603,auto-portee,Medium,3ans.,0.007406,0.757614
